In [1]:
import numpy as np
import pandas as pd
import re
pd.set_option('display.float_format', lambda x : '%.30f' % x)
pd.options.display.max_columns = None
pd.options.display.float_format = "{:,.3f}".format
import warnings
warnings.filterwarnings('ignore')
import datetime as dt

In [2]:
dataset = pd.read_excel("dump.xls", parse_dates = ["Date"])

In [3]:
# print(dataset.__len__())
# print(dataset.info())
(dataset)

,Account,Date,Transaction Type,No.,Name,Memo/Description,Split,Amount,Balance
0,Barter Account,04/01/2021,Payment,NaN,Website Sale,NaN,Debtors,3.150,3.150
1,Barter Account,04/01/2021,Payment,NaN,Website Sale,NaN,Debtors,6.220,9.370
2,Barter Account,05/01/2021,Payment,NaN,Website Sale,NaN,Debtors,5.510,14.880
3,Barter Account,06/01/2021,Payment,NaN,Website Sale,NaN,Debtors,3.830,18.710
4,Barter Account,06/01/2021,Payment,NaN,Website Sale,NaN,Debtors,2.980,21.690
...,...,...,...,...,...,...,...,...,...
22765,Not Specified,31/12/2021,Bill Payment (Cheque),Maria2188,Field Sports Publishing Inc,NaN,Creditors,0.000,0.000
22766,Not Specified,31/12/2021,Bill Payment (Cheque),Maria2192,Irons Brothers Ltd,NaN,Creditors,0.000,0.000
22767,Not Specified,31/12/2021,Bill Payment (Cheque),Maria2188,Reimer Johannsen,NaN,Creditors,0.000,0.000
22768,Not Specified,31/12/2021,Bill Payment (Cheque),Maria2188,Wbc,NaN,Creditors,0.000,0.000


In [4]:
dataset.loc[dataset.Date == 'Beginning Balance' ,"Date"] = "01/01/2021"
dataset["Date"] = pd.to_datetime(dataset["Date"], format = '%d/%m/%Y')
dataset.loc[dataset.Date == "01/01/2021","Date"]

1503    2021-01-01
1861    2021-01-01
1862    2021-01-01
3866    2021-01-01
3867    2021-01-01
           ...    
20518   2021-01-01
20519   2021-01-01
21494   2021-01-01
21735   2021-01-01
22248   2021-01-01
Name: Date, Length: 62, dtype: datetime64[ns]

In [5]:
date = "01/01/2021"
dt.datetime.strptime(date,"%d/%m/%Y")

datetime.datetime(2021, 1, 1, 0, 0)

In [6]:
bank_code = [
"Afex  Funds"
,"Barter Account"
,"Cash in Hand"
,"Cash on hand"
,"City forex"
,"Commercial card"
,"Commerzbank"
,"Fidelis Market Co"
,"KBR Forex payment bank"
,"Nat West 5869"
,"NatWest ****7810"
,"Natwest Saver ****7829"
,"Petty Cash"
]

In [7]:
dataset.columns = ['Account', 'Date', 'Transaction Type', 'No.', 'Name',
       'Memo/Description', 'Split', 'Amount', 'Balance']

In [8]:
dataset.columns

Index(['Account', 'Date', 'Transaction Type', 'No.', 'Name',
       'Memo/Description', 'Split', 'Amount', 'Balance'],
      dtype='object')

### bank_account

In [9]:
bank_account = dataset.loc[dataset['Account'].isin(bank_code)]
bank_account.__len__()
bank_account["No."].value_counts()

3388    23
3163    22
3350    22
2995    21
3194    21
        ..
1533     1
1442     1
1473     1
1478     1
3399     1
Name: No., Length: 2052, dtype: int64

In [10]:
prepayment_account = dataset.loc[dataset.loc[:,"Account"].isin(["Prepayment in GL"])]
prepayment_account.__len__()

0

### Accrual_Code

In [11]:
Accrual_Code = ["Accruals","Bad Debt Provision"]

In [12]:
Accrual_account = dataset.loc[dataset.Account.isin(Accrual_Code)]
Accrual_account.__len__()

121

### Revenue_code

In [13]:
Revenue_code = ["Discounts given",
"Dividend Income"
,"Sales of Product Income:Non UK Sales"
,"Sales of Product Income:Sales"
,"Sales of Product Income:UK Sales"]

In [14]:
Revenue_account = dataset.loc[dataset.Account.isin(Revenue_code)]
Revenue_account.__len__()

5

### bank_vs_revenue

In [15]:
bank_vs_revenue = pd.merge(bank_account,Revenue_account,on = "No.",how = "inner",suffixes=[" Bank"," Revenue"])

In [16]:
bank_vs_revenue

,Account Bank,Date Bank,Transaction Type Bank,No.,Name Bank,Memo/Description Bank,Split Bank,Amount Bank,Balance Bank,Account Revenue,Date Revenue,Transaction Type Revenue,Name Revenue,Memo/Description Revenue,Split Revenue,Amount Revenue,Balance Revenue
0,Commerzbank,2021-11-04,Journal Entry,3330,NaN,Dividend received from TEG,-Split-,"24,800.000","84,270.900",Dividend Income,2021-11-04,Journal Entry,NaN,Dividend received from TEG,-Split-,"24,800.000","24,800.000"
1,Commerzbank,2021-11-04,Journal Entry,3330,NaN,Dividend received from TEG,-Split-,"24,800.000","84,270.900",Dividend Income,2021-11-04,Journal Entry,NaN,Dividend received from TEG,-Split-,"24,800.000","49,600.000"


In [17]:
bank_vs_accrual = pd.merge(bank_account,Accrual_account,on ="No.",how = "inner",suffixes=[" Bank"," Accural"])

In [18]:
bank_vs_accrual

,Account Bank,Date Bank,Transaction Type Bank,No.,Name Bank,Memo/Description Bank,Split Bank,Amount Bank,Balance Bank,Account Accural,Date Accural,Transaction Type Accural,Name Accural,Memo/Description Accural,Split Accural,Amount Accural,Balance Accural
0,Barter Account,2021-01-04,Payment,NaN,Website Sale,NaN,Debtors,3.150,3.150,Accruals,2021-01-01,NaN,NaN,NaN,NaN,NaN,"30,050.000"
1,Barter Account,2021-01-04,Payment,NaN,Website Sale,NaN,Debtors,3.150,3.150,Accruals,2021-09-30,Bill,Blaser Group,Interest calculation for period 30 June - 30 S...,Creditors,"-32,504.130","111,494.450"
2,Barter Account,2021-01-04,Payment,NaN,Website Sale,NaN,Debtors,6.220,9.370,Accruals,2021-01-01,NaN,NaN,NaN,NaN,NaN,"30,050.000"
3,Barter Account,2021-01-04,Payment,NaN,Website Sale,NaN,Debtors,6.220,9.370,Accruals,2021-09-30,Bill,Blaser Group,Interest calculation for period 30 June - 30 S...,Creditors,"-32,504.130","111,494.450"
4,Barter Account,2021-01-05,Payment,NaN,Website Sale,NaN,Debtors,5.510,14.880,Accruals,2021-01-01,NaN,NaN,NaN,NaN,NaN,"30,050.000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,NatWest ****7810,2021-02-28,Journal Entry,3038,NaN,Saija Feb wages,-Split-,"-3,397.680","12,636.910",Accruals,2021-02-28,Journal Entry,NaN,Jerome Feb wages,-Split-,"3,152.740","54,564.620"
3773,NatWest ****7810,2021-03-01,Journal Entry,3047,NaN,Jerome Salary for Feb,-Split-,"-3,167.740","9,469.170",Accruals,2021-03-01,Journal Entry,NaN,Jerome Feb Salary,-Split-,"-3,167.740","52,693.880"
3774,NatWest ****7810,2021-12-16,Journal Entry,3369,NaN,Business rate payment March 21,-Split-,"-1,297.000","68,647.720",Accruals,2021-12-16,Journal Entry,NaN,Business rate payment,-Split-,"-3,891.000","103,617.640"
3775,NatWest ****7810,2021-12-16,Journal Entry,3369,NaN,Business rate payment Feb 21,-Split-,"-1,297.000","67,350.720",Accruals,2021-12-16,Journal Entry,NaN,Business rate payment,-Split-,"-3,891.000","103,617.640"


In [19]:
pattern = 

SyntaxError: invalid syntax (304381925.py, line 1)